In [1]:
import tensorflow as tf
print(tf. __version__)
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.losses import MeanSquaredError
# from tensorflow.keras import models
import pickle
import numpy as np
from tqdm import tqdm
tqdm.pandas()
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import Callback, EarlyStopping
import pandas as pd
import json
from sklearn.preprocessing import StandardScaler

#from google.colab import files
from sklearn.model_selection import train_test_split
from collections import Counter
#import smart_cond as sc

2.13.1


In [2]:
with open("./data/mimic_notext.pkl", "rb") as pfile:
    raw_data = pickle.load(pfile)
mimic = raw_data[0]
meta = raw_data[1]
train_ind = raw_data[2]
valid_ind = raw_data[3]
test_ind = raw_data[4]
data = mimic
oc = meta

In [3]:
ids = oc['SUBJECT_ID'].tolist()
labels = oc['in_hospital_sepsis'].tolist()

new_patient_ids = []
new_labels = []

for i in range(len(labels)):
  # print(i)
  if ids[i] in new_patient_ids:
    continue
  else:
    new_patient_ids.append(ids[i])
    new_labels.append(labels[i])

In [4]:
from sklearn.model_selection import train_test_split

x, x_test, y, y_test = train_test_split(new_patient_ids, new_labels, test_size=0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=1)
# train
train_ind = []

ts_ind = oc['ts_ind'].tolist()
# ids = ids

for i in range(len(ts_ind)):
  if ids[i] in x_train:
    train_ind.append(ts_ind[i])

# number of train patients
print(len(x_train))
# number of train instances
print(len(train_ind))
# to np.array
train_ind = np.array(train_ind)

test_ind = []

for i in range(len(ts_ind)):
  if ids[i] in x_test:
    test_ind.append(ts_ind[i])

# number of test patients
print(len(x_test))
# number of test instances
print(len(test_ind))
# to np.array
test_ind = np.array(test_ind)

valid_ind = []

for i in range(len(ts_ind)):
  if ids[i] in x_val:
    valid_ind.append(ts_ind[i])

# number of test patients
print(len(x_val))
# number of test instances
print(len(valid_ind))
# to np.array
valid_ind = np.array(valid_ind)

23587
31708
7371
9894
5897
7803


In [5]:
#dont need data preperation for classification task
"""
# # Read data.
# data_path = './../mimic_iii_preprocessed.pkl'
# data, oc, train_ind, valid_ind, test_ind = pickle.load(open(data_path, 'rb'))
# Filter labeled data in first 24h.
data = data.loc[data.ts_ind.isin(np.concatenate((train_ind, valid_ind, test_ind), axis=-1))]
data = data.loc[(data.hour>=0)&(data.hour<=24)]
oc = oc.loc[oc.ts_ind.isin(np.concatenate((train_ind, valid_ind, test_ind), axis=-1))]
# Fix age.
data.loc[(data.variable=='o:age')&(data.value>200), 'value'] = 91.4
# Get y and N.
y = np.array(oc.sort_values(by='ts_ind')['sepsis_label']).astype('float32')
N = data.ts_ind.max() + 1
# Get static data with mean fill and missingness indicator.
static_varis = ['o:age', 'o:gender']
ii = data.variable.isin(static_varis)
static_data = data.loc[ii]
data = data.loc[~ii]
def inv_list(l, start=0):
    d = {}
    for i in range(len(l)):
        d[l[i]] = i+start
    return d
static_var_to_ind = inv_list(static_varis)
D = len(static_varis)
demo = np.zeros((N, D))
for row in tqdm(static_data.itertuples()):
    demo[row.ts_ind, static_var_to_ind[row.variable]] = row.value
# Normalize static data.
means = demo.mean(axis=0, keepdims=True)
stds = demo.std(axis=0, keepdims=True)
stds = (stds==0)*1 + (stds!=0)*stds
demo = (demo-means)/stds
# Trim to max len.
#data = data.sample(frac=1)
data = data.groupby('ts_ind').head(880)
# Get N, V, var_to_ind.
N = data.ts_ind.max() + 1
varis = sorted(list(set(data.variable)))
V = len(varis)
def inv_list(l, start=0):
    d = {}
    for i in range(len(l)):
        d[l[i]] = i+start
    return d
var_to_ind = inv_list(varis, start=1)
data['vind'] = data.variable.map(var_to_ind)
data = data[['ts_ind', 'vind', 'hour', 'value']].sort_values(by=['ts_ind', 'vind', 'hour'])
# Add obs index.
data = data.sort_values(by=['ts_ind']).reset_index(drop=True)
data = data.reset_index().rename(columns={'index':'obs_ind'})
data = data.merge(data.groupby('ts_ind').agg({'obs_ind':'min'}).reset_index().rename(columns={ \
                                                            'obs_ind':'first_obs_ind'}), on='ts_ind')
data['obs_ind'] = data['obs_ind'] - data['first_obs_ind']
# Find max_len.
max_len = data.obs_ind.max()+1
print ('max_len', max_len)
# Generate times_ip and values_ip matrices.
times_inp = np.zeros((N, max_len), dtype='float32')
values_inp = np.zeros((N, max_len), dtype='float32')
varis_inp = np.zeros((N, max_len), dtype='int32')
for row in tqdm(data.itertuples()):
    ts_ind = row.ts_ind
    l = row.obs_ind
    times_inp[ts_ind, l] = row.hour
    values_inp[ts_ind, l] = row.value
    varis_inp[ts_ind, l] = row.vind
data.drop(columns=['obs_ind', 'first_obs_ind'], inplace=True)
# Generate 3 sets of inputs and outputs.
train_ip = [ip[train_ind] for ip in [demo, times_inp, values_inp, varis_inp]]
valid_ip = [ip[valid_ind] for ip in [demo, times_inp, values_inp, varis_inp]]
test_ip = [ip[test_ind] for ip in [demo, times_inp, values_inp, varis_inp]]
del times_inp, values_inp, varis_inp
train_op = y[train_ind]
valid_op = y[valid_ind]
test_op = y[test_ind]
del y
"""

"\n# # Read data.\n# data_path = './../mimic_iii_preprocessed.pkl'\n# data, oc, train_ind, valid_ind, test_ind = pickle.load(open(data_path, 'rb'))\n# Filter labeled data in first 24h.\ndata = data.loc[data.ts_ind.isin(np.concatenate((train_ind, valid_ind, test_ind), axis=-1))]\ndata = data.loc[(data.hour>=0)&(data.hour<=24)]\noc = oc.loc[oc.ts_ind.isin(np.concatenate((train_ind, valid_ind, test_ind), axis=-1))]\n# Fix age.\ndata.loc[(data.variable=='o:age')&(data.value>200), 'value'] = 91.4\n# Get y and N.\ny = np.array(oc.sort_values(by='ts_ind')['sepsis_label']).astype('float32')\nN = data.ts_ind.max() + 1\n# Get static data with mean fill and missingness indicator.\nstatic_varis = ['o:age', 'o:gender']\nii = data.variable.isin(static_varis)\nstatic_data = data.loc[ii]\ndata = data.loc[~ii]\ndef inv_list(l, start=0):\n    d = {}\n    for i in range(len(l)):\n        d[l[i]] = i+start\n    return d\nstatic_var_to_ind = inv_list(static_varis)\nD = len(static_varis)\ndemo = np.zeros((N

In [6]:
mape = tf.keras.losses.MeanAbsolutePercentageError()
def get_res(y_true, y_pred):
    precision, recall, thresholds = precision_recall_curve(y_true, y_pred)
    pr_auc = auc(recall, precision)
    minrp = np.minimum(precision, recall).max()
    roc_auc = roc_auc_score(y_true, y_pred)
    return [roc_auc, pr_auc, minrp]

########################################################################################################
########################################################################################################
#class_weights = compute_class_weight(class_weight='balanced', classes=[0,1], y=train_op)
def mortality_loss(y_true, y_pred):
    sample_weights = (1-y_true)*class_weights[0] + y_true*class_weights[1]
    bce = K.binary_crossentropy(y_true, y_pred)
    return K.mean(sample_weights*bce, axis=-1)
########################################################################################################
########################################################################################################

# var_weights = np.sum(fore_train_op[:, V:], axis=0)
# var_weights[var_weights==0] = var_weights.max()
# var_weights = var_weights.max()/var_weights
# var_weights = var_weights.reshape((1, V))
def forecast_loss(y_true, y_pred):
    return K.sum(y_true[:,V:]*(y_true[:,:V]-y_pred)**2, axis=-1)

def mape_fore(y_true, y_pred):
    truth = y_true[:,V:]
    pred = y_pred
    return mape(truth, pred)

                                          
def get_min_loss(weight):
    def min_loss(y_true, y_pred):
        return weight*y_pred
    return min_loss

class CustomCallback(Callback):
    def __init__(self, validation_data, batch_size):
        self.val_x, self.val_y = validation_data
        self.batch_size = batch_size
        super(Callback, self).__init__()

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.val_x, verbose=0, batch_size=self.batch_size)
        if type(y_pred)==type([]):
            y_pred = y_pred[0]
        precision, recall, thresholds = precision_recall_curve(self.val_y, y_pred)
        pr_auc = auc(recall, precision)
        roc_auc = roc_auc_score(self.val_y, y_pred)
        logs['custom_metric'] = pr_auc + roc_auc
        print ('val_aucs:', pr_auc, roc_auc)

In [7]:
import tensorflow as tf
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Embedding, Activation, Dropout, Softmax, Layer, InputSpec, Input, Dense, Lambda, TimeDistributed, Concatenate, Add
from tensorflow.keras import initializers, regularizers, constraints, Model
from tensorflow.python.keras.utils import tf_utils
from tensorflow.python.ops import array_ops
from tensorflow import nn
import smart_cond_mod as sc


class CVE(Layer):
    def __init__(self, hid_units, output_dim):
        self.hid_units = hid_units
        self.output_dim = output_dim
        super(CVE, self).__init__()

    def build(self, input_shape):
        self.W1 = self.add_weight(name='CVE_W1',
                            shape=(1, self.hid_units),
                            initializer='glorot_uniform',
                            trainable=True)
        self.b1 = self.add_weight(name='CVE_b1',
                            shape=(self.hid_units,),
                            initializer='zeros',
                            trainable=True)
        self.W2 = self.add_weight(name='CVE_W2',
                            shape=(self.hid_units, self.output_dim),
                            initializer='glorot_uniform',
                            trainable=True)
        super(CVE, self).build(input_shape)

    def call(self, x):
        x = K.expand_dims(x, axis=-1)
        x = K.dot(K.tanh(K.bias_add(K.dot(x, self.W1), self.b1)), self.W2)
        return x

    def compute_output_shape(self, input_shape):
        return input_shape + (self.output_dim,)


class Attention(Layer):

    def __init__(self, hid_dim):
        self.hid_dim = hid_dim
        super(Attention, self).__init__()

    def build(self, input_shape):
        d = input_shape.as_list()[-1]
        self.W = self.add_weight(shape=(d, self.hid_dim), name='Att_W',
                                 initializer='glorot_uniform',
                                 trainable=True)
        self.b = self.add_weight(shape=(self.hid_dim,), name='Att_b',
                                 initializer='zeros',
                                 trainable=True)
        self.u = self.add_weight(shape=(self.hid_dim,1), name='Att_u',
                                 initializer='glorot_uniform',
                                 trainable=True)
        super(Attention, self).build(input_shape)

    def call(self, x, mask, mask_value=-1e30):
        attn_weights = K.dot(K.tanh(K.bias_add(K.dot(x,self.W), self.b)), self.u)
        mask = K.expand_dims(mask, axis=-1)
        attn_weights = mask*attn_weights + (1-mask)*mask_value
        attn_weights = K.softmax(attn_weights, axis=-2)
        return attn_weights

    def compute_output_shape(self, input_shape):
        return input_shape[:-1] + (1,)


class Transformer(Layer):

    def __init__(self, N=2, h=8, dk=None, dv=None, dff=None, dropout=0):
        self.N, self.h, self.dk, self.dv, self.dff, self.dropout = N, h, dk, dv, dff, dropout
        self.epsilon = K.epsilon() * K.epsilon()
        super(Transformer, self).__init__()

    def build(self, input_shape):
        d = input_shape.as_list()[-1]
        if self.dk==None:
            self.dk = d//self.h
        if self.dv==None:
            self.dv = d//self.h
        if self.dff==None:
            self.dff = 2*d
        self.Wq = self.add_weight(shape=(self.N, self.h, d, self.dk), name='Wq',
                                 initializer='glorot_uniform', trainable=True)
        self.Wk = self.add_weight(shape=(self.N, self.h, d, self.dk), name='Wk',
                                 initializer='glorot_uniform', trainable=True)
        self.Wv = self.add_weight(shape=(self.N, self.h, d, self.dv), name='Wv',
                                 initializer='glorot_uniform', trainable=True)
        self.Wo = self.add_weight(shape=(self.N, self.dv*self.h, d), name='Wo',
                                 initializer='glorot_uniform', trainable=True)
        self.W1 = self.add_weight(shape=(self.N, d, self.dff), name='W1',
                                 initializer='glorot_uniform', trainable=True)
        self.b1 = self.add_weight(shape=(self.N, self.dff), name='b1',
                                 initializer='zeros', trainable=True)
        self.W2 = self.add_weight(shape=(self.N, self.dff, d), name='W2',
                                 initializer='glorot_uniform', trainable=True)
        self.b2 = self.add_weight(shape=(self.N, d), name='b2',
                                 initializer='zeros', trainable=True)
        self.gamma = self.add_weight(shape=(2*self.N,), name='gamma',
                                 initializer='ones', trainable=True)
        self.beta = self.add_weight(shape=(2*self.N,), name='beta',
                                 initializer='zeros', trainable=True)
        super(Transformer, self).build(input_shape)

    def call(self, x, mask, mask_value=-1e-30):
        mask = K.expand_dims(mask, axis=-2)
        for i in range(self.N):
            # MHA
            mha_ops = []
            for j in range(self.h):
                q = K.dot(x, self.Wq[i,j,:,:])
                k = K.permute_dimensions(K.dot(x, self.Wk[i,j,:,:]), (0,2,1))
                v = K.dot(x, self.Wv[i,j,:,:])
                A = K.batch_dot(q,k)
                # Mask unobserved steps.
                A = mask*A + (1-mask)*mask_value
                # Mask for attention dropout.
                def dropped_A():
                    dp_mask = K.cast((K.random_uniform(shape=array_ops.shape(A))>=self.dropout), K.floatx())
                    return A*dp_mask + (1-dp_mask)*mask_value
                A = sc.smart_cond(K.learning_phase(), dropped_A, lambda: array_ops.identity(A))
                A = K.softmax(A, axis=-1)
                mha_ops.append(K.batch_dot(A,v))
            conc = K.concatenate(mha_ops, axis=-1)
            proj = K.dot(conc, self.Wo[i,:,:])
            # Dropout.
            proj = sc.smart_cond(K.learning_phase(), lambda: array_ops.identity(nn.dropout(proj, rate=self.dropout)),\
                                       lambda: array_ops.identity(proj))
            # Add & LN
            x = x+proj
            mean = K.mean(x, axis=-1, keepdims=True)
            variance = K.mean(K.square(x - mean), axis=-1, keepdims=True)
            std = K.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x*self.gamma[2*i] + self.beta[2*i]
            # FFN
            ffn_op = K.bias_add(K.dot(K.relu(K.bias_add(K.dot(x, self.W1[i,:,:]), self.b1[i,:])),
                           self.W2[i,:,:]), self.b2[i,:,])
            # Dropout.
            ffn_op = sc.smart_cond(K.learning_phase(), lambda: array_ops.identity(nn.dropout(ffn_op, rate=self.dropout)),\
                                       lambda: array_ops.identity(ffn_op))
            # Add & LN
            x = x+ffn_op
            mean = K.mean(x, axis=-1, keepdims=True)
            variance = K.mean(K.square(x - mean), axis=-1, keepdims=True)
            std = K.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x*self.gamma[2*i+1] + self.beta[2*i+1]
        return x

    def compute_output_shape(self, input_shape):
        return input_shape


def build_strats(D, max_len, V, d, N, he, dropout, forecast=False):
    demo = Input(shape=(D,))
    demo_enc = Dense(2*d, activation='tanh')(demo)
    demo_enc = Dense(d, activation='tanh')(demo_enc)
    varis = Input(shape=(max_len,))
    values = Input(shape=(max_len,))
    times = Input(shape=(max_len,))
    varis_emb = Embedding(V+1, d)(varis)
    cve_units = int(np.sqrt(d))
    values_emb = CVE(cve_units, d)(values)
    times_emb = CVE(cve_units, d)(times)
    comb_emb = Add()([varis_emb, values_emb, times_emb]) # b, L, d
#     demo_enc = Lambda(lambda x:K.expand_dims(x, axis=-2))(demo_enc) # b, 1, d
#     comb_emb = Concatenate(axis=-2)([demo_enc, comb_emb]) # b, L+1, d
    mask = Lambda(lambda x:K.clip(x,0,1))(varis) # b, L
#     mask = Lambda(lambda x:K.concatenate((K.ones_like(x)[:,0:1], x), axis=-1))(mask) # b, L+1
    cont_emb = Transformer(N, he, dk=None, dv=None, dff=None, dropout=dropout)(comb_emb, mask=mask)
    attn_weights = Attention(2*d)(cont_emb, mask=mask)
    fused_emb = Lambda(lambda x:K.sum(x[0]*x[1], axis=-2))([cont_emb, attn_weights])
    conc = Concatenate(axis=-1)([fused_emb, demo_enc])
    fore_op = Dense(V)(conc)
    op = Dense(1, activation='sigmoid')(fore_op)
    model = Model([demo, times, values, varis], op)
    if forecast:
        fore_model = Model([demo, times, values, varis], fore_op)
        return [model, fore_model]
    return model

# To tune:
# 1. Transformer parameters. (N, h, dropout)
# 2. Normalization

## get forecast on test patients

### Load test data into matrices

In [8]:
forecast_hours = 4
pred_window = 2 # hours that the output vector represents. 1 because i want to learn to predict 1 hour many times, 2 to test standard strats config
obs_windows = [4]

In [9]:
# take test patients.
data = data.merge(oc[['ts_ind', 'SUBJECT_ID']], on='ts_ind', how='left')
test_sub = oc.loc[oc.ts_ind.isin(test_ind[:10])].SUBJECT_ID.unique()
data = data.loc[data.SUBJECT_ID.isin(test_sub[:10])]
oc = oc.loc[oc.SUBJECT_ID.isin(test_sub[:10])]
data.drop(columns=['SUBJECT_ID'], inplace=True)
# Fix age.
data.loc[(data.variable=='Age')&(data.value>200), 'value'] = 91.4
# Get static data with mean fill and missingness indicator.
static_varis = ['Age', 'Gender']
ii = data.variable.isin(static_varis)
static_data = data.loc[ii]
data = data.loc[~ii]
def inv_list(l, start=0):
    d = {}
    for i in range(len(l)):
        d[l[i]] = i+start
    return d
static_var_to_ind = inv_list(static_varis)
D = len(static_varis)
N = data.ts_ind.max()+1
demo = np.zeros((N, D))
for row in tqdm(static_data.itertuples()):
    demo[row.ts_ind, static_var_to_ind[row.variable]] = row.value
# Normalize static data.
means = demo.mean(axis=0, keepdims=True)
stds = demo.std(axis=0, keepdims=True)
stds = (stds==0)*1 + (stds!=0)*stds
demo = (demo-means)/stds

# Get variable indices.
varis = sorted(list(set(data.variable)))
V = len(varis)
var_to_ind = inv_list(varis, start=1)
data['vind'] = data.variable.map(var_to_ind)
data = data[['ts_ind', 'vind', 'hour', 'value']].sort_values(by=['ts_ind', 'vind', 'hour'])
# Find max_len.
fore_max_len = 880
# Get forecast inputs and outputs.
fore_times_ip = []
fore_values_ip = []
fore_varis_ip = []
fore_op = []
fore_inds = []
def f(x):
    mask = [0 for i in range(V)]
    values = [0 for i in range(V)]
    for vv in x:
        v = int(vv[0])-1
        mask[v] = 1
        values[v] = vv[1]
    return values+mask
def pad(x):
    return x+[0]*(V+V*forecast_hours-len(x))

for w in tqdm(obs_windows):
    pred_data = data.loc[(data.hour>=w)&(data.hour<=w+pred_window)]
    pred_data = pred_data.groupby(['ts_ind', 'vind']).agg({'value':'first'}).reset_index()
    pred_data['vind_value'] = pred_data[['vind', 'value']].values.tolist()
    pred_data = pred_data.groupby('ts_ind').agg({'vind_value':list}).reset_index()
    pred_data['vind_value'] = pred_data['vind_value'].apply(f)
    obs_data = data.loc[(data.hour<w)&(data.hour>=w-24)]
    obs_data = obs_data.loc[obs_data.ts_ind.isin(pred_data.ts_ind)]
    obs_data = obs_data.groupby('ts_ind').head(fore_max_len)
    obs_data = obs_data.groupby('ts_ind').agg({'vind':list, 'hour':list, 'value':list}).reset_index()
    obs_data = obs_data.merge(pred_data, on='ts_ind')
    for col in ['vind', 'hour', 'value']:
        obs_data[col] = obs_data[col].apply(pad)
    fore_op.append(np.array(list(obs_data.vind_value)))
    fore_inds.append(np.array(list(obs_data.ts_ind)))
    fore_times_ip.append(np.array(list(obs_data.hour)))
    fore_values_ip.append(np.array(list(obs_data.value)))
    fore_varis_ip.append(np.array(list(obs_data.vind)))

del data
fore_times_ip = np.concatenate(fore_times_ip, axis=0)
fore_values_ip = np.concatenate(fore_values_ip, axis=0)
fore_varis_ip = np.concatenate(fore_varis_ip, axis=0)
fore_op = np.concatenate(fore_op, axis=0)
fore_inds = np.concatenate(fore_inds, axis=0)
fore_demo = demo[fore_inds]

fore_test_ip = [fore_demo, fore_times_ip, fore_values_ip, fore_varis_ip]
fore_test_op = fore_op

# release RAM
del fore_times_ip, fore_values_ip, fore_varis_ip, demo, fore_demo

20it [00:00, 107546.26it/s]
100%|██████████| 1/1 [00:00<00:00, 100.32it/s]


In [10]:
x = fore_test_ip[3]
print(len(x[0]),x[0], len(x))

415 [ 6  8 11 17 18 19 21 22 22 22 22 22 22 23 23 23 28 28 29 29 30 30 33 34
 35 35 35 35 35 35 37 39 47 47 47 47 47 47 48 49 50 51 57 57 57 57 57 57
 64 65 66 69 70 71 71 71 71 72 72 72 72 72 72 74 77 77 77 79 79 80 81  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  

In [11]:
def repeat_prediction(first_input, y_preds, relative_time, max_seq_len, len_vari=V):
    
    # build new input with prediction added as "newest data"
    # take highest "time" and add number of forecasting steps -> time of new forecast
    preds_time = (first_input[1].max(axis=1, keepdims=True))+relative_time
    #print("predstime", preds_time)
    # all features are predicted at that same "new" time
    preds_time = np.repeat(preds_time, len_vari, axis=1)
    #print("predstime", preds_time)
    # each variable is predicted
    preds_varid = np.linspace(1, len_vari, num=V)
    #print("varid", preds_varid)
    # repeat for each time already predicted
    preds_varid = np.tile(preds_varid, (len(preds_time), 1))
    #print("varid", preds_varid)
    # take the original demographics, as it does not change
    preds_demo = first_input[0]
    new_data = [preds_demo, preds_time, y_preds, preds_varid]
    
    
    new_fore_ip = [first_input[0]]
    if relative_time == 0:
        for i in range(1,4):
            new_fore_ip.append(np.append(first_input[i][:,:len_vari], new_data[i], axis=1)) #only take existing values w/o padding and combine with prediction
        print(len(new_fore_ip[1][0]))
    else:
        for i in range(1,4):
            new_fore_ip.append(np.append(first_input[i][:,:len_vari*(relative_time+1)], new_data[i], axis=1)) #only take existing values w/o padding and combine with prediction
        print(len(new_fore_ip[1][0]))


    f = np.zeros((len(new_fore_ip[1]),(max_seq_len-len(new_fore_ip[1][0])))) # padding
    for i in range(1,4):
        new_fore_ip[i] = np.append(new_fore_ip[i],f, axis=1) # add padding
    
    #return to predict again, this time with new data, containing previous predictions and so on
    return new_fore_ip, new_data

### get test `y_preds`

In [12]:
# for matplotlib purposes
start_ip = fore_test_ip

In [18]:
l = start_ip[1]

In [13]:
fore_test_ip = start_ip

In [14]:

fore_savepath = './STraTS_4-15_masked_data_transformer_testt'

lr, batch_size, samples_per_epoch, patience = 0.0005, 32, len(fore_test_op), 5
d, N, he, dropout = 50, 2, 4, 0.2
model, fore_model =  build_strats(D, V+(V*forecast_hours), V, d, N, he, dropout, forecast=True)
print(fore_model.summary())
# fore_model.compile(loss=forecast_loss, optimizer=Adam(lr))
lossfunction = forecast_loss
opt = tf.keras.optimizers.Adam(lr)
fore_model.compile(loss=lossfunction, optimizer=opt)

# initialize checkpoint manager
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=fore_model)
manager = tf.train.CheckpointManager(ckpt, f'{fore_savepath}', max_to_keep=3)
ckpt.restore(manager.latest_checkpoint)
if manager.latest_checkpoint:
    print("Restored from {}".format(manager.latest_checkpoint))
else:
    print("Initializing from scratch.")


# forecasting
for i in range(0, forecast_hours):
    # predict and save prediction
    test_y_preds = fore_model.predict(fore_test_ip)

    # during 1st run, initialize cumulated predictions
    if i == 0:
        predictions = test_y_preds
        
    # during following runs, concatenate predictions
    else:
        predictions = np.concatenate((predictions,test_y_preds), axis=1)
        
    # build new input consisting of original input + hourly predictions appended
    fore_test_ip, n_data_ = repeat_prediction(fore_test_ip,test_y_preds, i, max_seq_len=V+(V*forecast_hours))  
    
    # add V to sequence length as it is the length of prediction, to keep track

"""
hours = []
max_hours = []
# get hours
for time in start_ip[1]:
  hour = max(time)
  max_hours.append(hour)

  for obs_window in obs_windows:
    if hour < obs_window:
      hour = obs_window
      break

  hours.append(hour)

# get patient ids
test_patient_ids = []
test_sepsis_labels = []

# sub_ids = oc['SUBJECT_ID'].tolist()
# ts_ind = oc['ts_ind'].tolist()
# sepsis = oc['in_hospital_sepsis'].tolist()

# for ind in val_inds:
#    for i in range(len(sub_ids)):
#      if ts_ind[i] == ind:
#        val_sepsis_labels.append(sepsis[i])
#        val_patient_ids.append(sub_ids[i])
#        break

for ind in tqdm(fore_inds):
  test_sepsis_labels.append(np.unique(oc[oc['ts_ind']==ind]['in_hospital_sepsis']).item())
  test_patient_ids.append(np.unique(oc[oc['ts_ind']==ind]['SUBJECT_ID']).item())

test_data = pd.DataFrame(
    {'ts_ind': fore_inds,
     'obs_window': hours,
     'SUBJECT_ID': test_patient_ids,
     'sepsis_label': test_sepsis_labels,
     'forecasting_pred': pd.Series(predictions.tolist()),
     'forecasting_test_op': pd.Series(fore_op.tolist())
    })

# dump to pkl
pickle.dump([test_data, var_to_ind], open('STraTS_4-15_masked_transformer_4-24_forecast','wb'))"""

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 415)]                0         []                            
                                                                                                  
 input_3 (InputLayer)        [(None, 415)]                0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 415)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 415, 50)              4200      ['input_2[0][0]']             
                                                                                            

None
Initializing from scratch.
1/1 [==============================] - 0s 333ms/step
166
1/1 [==============================] - 0s 309ms/step
249
1/1 [==============================] - 0s 35ms/step
332
1/1 [==============================] - 0s 36ms/step
415


"\nhours = []\nmax_hours = []\n# get hours\nfor time in start_ip[1]:\n  hour = max(time)\n  max_hours.append(hour)\n\n  for obs_window in obs_windows:\n    if hour < obs_window:\n      hour = obs_window\n      break\n\n  hours.append(hour)\n\n# get patient ids\ntest_patient_ids = []\ntest_sepsis_labels = []\n\n# sub_ids = oc['SUBJECT_ID'].tolist()\n# ts_ind = oc['ts_ind'].tolist()\n# sepsis = oc['in_hospital_sepsis'].tolist()\n\n# for ind in val_inds:\n#    for i in range(len(sub_ids)):\n#      if ts_ind[i] == ind:\n#        val_sepsis_labels.append(sepsis[i])\n#        val_patient_ids.append(sub_ids[i])\n#        break\n\nfor ind in tqdm(fore_inds):\n  test_sepsis_labels.append(np.unique(oc[oc['ts_ind']==ind]['in_hospital_sepsis']).item())\n  test_patient_ids.append(np.unique(oc[oc['ts_ind']==ind]['SUBJECT_ID']).item())\n\ntest_data = pd.DataFrame(\n    {'ts_ind': fore_inds,\n     'obs_window': hours,\n     'SUBJECT_ID': test_patient_ids,\n     'sepsis_label': test_sepsis_labels,\n   

In [15]:

#[preds_demo, preds_time, y_preds, preds_varid]
i = 2
#h = fore_test_ip[i]
#h = pd.DataFrame(h)
#print(h.iloc[0][100:200])
x = fore_test_ip[i]
x = pd.DataFrame(x)
print(x.iloc[0][150:200])
#y = n_data_[i]
#print(len(y[0]),y[0], len(y))

150   -0.530090
151    1.575771
152    0.858949
153   -0.310377
154    0.428438
155   -1.133890
156    0.864915
157    0.661443
158   -0.606896
159    0.347655
160    0.366879
161   -0.426487
162    0.000974
163    0.140145
164    1.115678
165   -0.959354
166   -0.044851
167    0.060840
168    0.682699
169   -1.439977
170   -0.308817
171    0.140238
172   -0.983397
173   -0.519898
174   -0.360838
175   -0.086720
176   -0.202925
177   -1.100451
178    0.866807
179    0.132619
180    1.706447
181    0.021675
182   -0.018953
183   -0.226438
184   -0.032558
185   -0.474229
186   -0.417515
187   -0.727073
188   -0.412518
189   -0.626417
190   -0.124638
191   -0.408152
192   -0.237336
193    1.600364
194    0.686340
195   -1.135021
196    0.510228
197   -0.529614
198   -0.279778
199   -0.089563
Name: 0, dtype: float64


In [102]:
h

,0,1,2,3,4,5,6,7,8,9,...,2862,2863,2864,2865,2866,2867,2868,2869,2870,2871
0,0.334275,-0.512236,0.406903,-1.019767,1.256110,-0.676131,0.762475,0.861151,0.861151,0.519934,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.327616,-0.299017,-0.266831,1.181628,0.584069,-0.551301,0.022152,-0.475407,0.052665,-0.286301,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.325079,-0.325079,-0.325079,-0.776693,-1.049667,-1.049667,-1.459128,0.657743,0.679176,0.679176,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.084480,0.269058,1.368780,-0.594784,-0.409003,-0.776693,0.679176,0.515222,1.058598,-0.769985,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.664902,0.464381,2.907784,-1.377245,-2.445678,0.325597,-0.517001,0.532169,0.532169,1.611830,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.000000,0.000000,0.000000,0.762475,0.656421,0.519934,0.451690,-0.094258,0.679176,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2.041950,0.246960,-0.094258,0.042229,0.792908,-0.298988,0.679176,0.679176,0.515222,0.515222,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,-0.236197,0.001778,0.084480,-0.746624,0.022152,-0.286301,-0.342221,-0.517001,-0.341712,-0.389053,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,-0.517001,0.929395,1.202369,0.246960,-0.026014,-0.162501,-0.913180,-1.186154,-1.459128,0.679176,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.588177,0.451690,0.656421,0.679176,0.515222,1.058598,0.496954,0.006264,0.387912,-0.847678,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
print(len(predictions[0]))
4*83

332


332

In [18]:
fore_savepath = './STraTS_20-124_masked_transformer'

lr, batch_size, samples_per_epoch, patience = 0.0005, 32, len(fore_test_op), 5
d, N, he, dropout = 50, 2, 4, 0.2
model, fore_model =  build_strats(D, fore_max_len+V*forecast_hours, V, d, N, he, dropout, forecast=True)
print(fore_model.summary())
# fore_model.compile(loss=forecast_loss, optimizer=Adam(lr))
lossfunction = forecast_loss
opt = tf.keras.optimizers.Adam(lr)
fore_model.compile(loss=lossfunction, optimizer=opt)

# initialize checkpoint manager
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=fore_model)
manager = tf.train.CheckpointManager(ckpt, f'{fore_savepath}', max_to_keep=3)
ckpt.restore(manager.latest_checkpoint)
if manager.latest_checkpoint:
    print("Restored from {}".format(manager.latest_checkpoint))
else:
    print("Initializing from scratch.")
# for matplotlib purposes
start_ip = fore_test_ip
# set initial seq_len so we can "count" where we are during the repeated forecasting process
seq_len = 880
# forecasting
for i in range(0, forecast_hours):
    # predict and save prediction
    test_y_preds = fore_model.predict(fore_test_ip)

    # during 1st run, initialize cumulated predictions
    if i == 0:
        predictions = test_y_preds
        
    # during following runs, concatenate predictions
    else:
        predictions = np.concatenate((predictions,test_y_preds), axis=1)
        
    # build new input consisting of original input + hourly predictions appended
    fore_test_ip = repeat_prediction(fore_test_ip,test_y_preds, i, seq_len, max_seq_len=fore_max_len+V*forecast_hours)  
    
    # add V to sequence length as it is the length of prediction, to keep track
    seq_len+=V 

hours = []
max_hours = []
# get hours
for time in start_ip[1]:
  hour = max(time)
  max_hours.append(hour)

  for obs_window in obs_windows:
    if hour < obs_window:
      hour = obs_window
      break

  hours.append(hour)

# get patient ids
test_patient_ids = []
test_sepsis_labels = []

# sub_ids = oc['SUBJECT_ID'].tolist()
# ts_ind = oc['ts_ind'].tolist()
# sepsis = oc['in_hospital_sepsis'].tolist()

# for ind in val_inds:
#    for i in range(len(sub_ids)):
#      if ts_ind[i] == ind:
#        val_sepsis_labels.append(sepsis[i])
#        val_patient_ids.append(sub_ids[i])
#        break

for ind in tqdm(fore_inds):
  test_sepsis_labels.append(np.unique(oc[oc['ts_ind']==ind]['in_hospital_sepsis']).item())
  test_patient_ids.append(np.unique(oc[oc['ts_ind']==ind]['SUBJECT_ID']).item())

test_data = pd.DataFrame(
    {'ts_ind': fore_inds,
     'obs_window': hours,
     'SUBJECT_ID': test_patient_ids,
     'sepsis_label': test_sepsis_labels,
     'forecasting_pred': pd.Series(predictions.tolist()),
     'forecasting_test_op': pd.Series(fore_op.tolist())
    })

# dump to pkl
pickle.dump([test_data, var_to_ind], open('STraTS_20-124_masked_transformer_4-24-2_forecast.pkl','wb'))

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_22 (InputLayer)       [(None, 4072)]               0         []                            
                                                                                                  
 input_23 (InputLayer)       [(None, 4072)]               0         []                            
                                                                                                  
 input_24 (InputLayer)       [(None, 4072)]               0         []                            
                                                                                                  
 embedding_5 (Embedding)     (None, 4072, 50)             6700      ['input_22[0][0]']            
                                                                                           

100%|██████████| 9657/9657 [00:05<00:00, 1641.71it/s]


### `var_to_ind` mapping

In [ ]:
var_to_ind

In [ ]:
hours = []
max_hours = []
# get hours
for time in start_ip[1]:
  hour = max(time)
  max_hours.append(hour)

  for obs_window in obs_windows:
    if hour < obs_window:
      hour = obs_window
      break

  hours.append(hour)

### patient ids, sepsis labels

In [50]:
# get patient ids
test_patient_ids = []
test_sepsis_labels = []

# sub_ids = oc['SUBJECT_ID'].tolist()
# ts_ind = oc['ts_ind'].tolist()
# sepsis = oc['in_hospital_sepsis'].tolist()

# for ind in val_inds:
#    for i in range(len(sub_ids)):
#      if ts_ind[i] == ind:
#        val_sepsis_labels.append(sepsis[i])
#        val_patient_ids.append(sub_ids[i])
#        break

for ind in tqdm(fore_inds):
  test_sepsis_labels.append(np.unique(oc[oc['ts_ind']==ind]['in_hospital_sepsis']).item())
  test_patient_ids.append(np.unique(oc[oc['ts_ind']==ind]['SUBJECT_ID']).item())

100%|██████████| 3923/3923 [15:12<00:00,  4.30it/s]


In [51]:
test_patient_ids

[200011.0,
 200079.0,
 200224.0,
 200232.0,
 200281.0,
 200297.0,
 200298.0,
 200364.0,
 200379.0,
 200386.0,
 200387.0,
 200392.0,
 200430.0,
 200450.0,
 200509.0,
 200557.0,
 200566.0,
 200637.0,
 200661.0,
 200677.0,
 200686.0,
 200748.0,
 200754.0,
 200824.0,
 200845.0,
 200873.0,
 200893.0,
 200902.0,
 200931.0,
 200932.0,
 200934.0,
 200947.0,
 200961.0,
 200993.0,
 201001.0,
 201018.0,
 201020.0,
 201024.0,
 201028.0,
 201043.0,
 201160.0,
 201164.0,
 201173.0,
 201184.0,
 201211.0,
 201220.0,
 201261.0,
 201264.0,
 201271.0,
 201326.0,
 201371.0,
 201400.0,
 201401.0,
 201444.0,
 201466.0,
 201483.0,
 201543.0,
 201569.0,
 201584.0,
 201585.0,
 201587.0,
 201756.0,
 201866.0,
 201876.0,
 201877.0,
 201951.0,
 201970.0,
 201982.0,
 202010.0,
 202019.0,
 202032.0,
 202057.0,
 202063.0,
 202129.0,
 202211.0,
 202214.0,
 202237.0,
 202248.0,
 202256.0,
 202268.0,
 202287.0,
 202291.0,
 202293.0,
 202304.0,
 202309.0,
 202351.0,
 202368.0,
 202385.0,
 202403.0,
 202414.0,
 202419.0,

### output to pkl

In [54]:
test_data = pd.DataFrame(
    {'ts_ind': fore_inds,
     'obs_window': hours,
     'SUBJECT_ID': test_patient_ids,
     'sepsis_label': test_sepsis_labels,
     'forecasting_pred': pd.Series(predictions.tolist()),
     'forecasting_test_op': pd.Series(fore_op.tolist())
    })

In [55]:
test_data

,ts_ind,obs_window,SUBJECT_ID,sepsis_label,forecasting_pred,forecasting_test_op
0,5,4,200011.0,1,"[1.056995153427124, -0.6347951292991638, -0.50...","[0.8400016372490969, -0.8902682310165556, -1.5..."
1,38,4,200079.0,0,"[-1.806349515914917, -0.38487932085990906, -0....","[-0.0195401924382678, 0.0097278901485559, 0.46..."
2,98,4,200224.0,0,"[0.9422867894172668, 0.5341206789016724, 0.036...","[1.0171875777582011, -0.8952046714397234, 1.33..."
3,103,4,200232.0,0,"[-0.37082815170288086, -0.16450455784797668, 0...","[-0.2787221349873852, 0.9900714985358582, 1.19..."
4,121,4,200281.0,0,"[-0.14169426262378693, 1.6773834228515625, -0....","[0.2396417501108494, 4.45897965129093, -0.2576..."
...,...,...,...,...,...,...
3918,43327,4,299861.0,0,"[-1.1628892421722412, 1.0943489074707031, -0.9...","[-1.833813790282089, 1.0654825453348815, -1.56..."
3919,43348,4,299911.0,0,"[0.1815841943025589, 0.9949281215667725, 0.455...","[-0.0195401924382678, 1.9704151069231617, 0.17..."
3920,43367,4,299955.0,1,"[0.37157684564590454, -0.3972075581550598, 0.7...","[1.0716606948959595, -0.6428363572216266, 0.97..."
3921,43378,4,299976.0,0,"[-0.459381639957428, 1.2800699472427368, -0.71...","[-0.0519379352569075, 0.6130162645407418, -0.3..."


In [56]:
# dump to pkl
pickle.dump([test_data, var_to_ind], open('strats_base_test.pkl','wb'))